<img src="logo.png">

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = [12, 12]
np.random.seed(42)

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
make_blobs?

In [ ]:
K = 3
n_muestras = 5000

centroides = [(24, 30), (52, 35), (35, 60)]
X, clases_reales = make_blobs(n_samples=n_muestras, n_features=2, cluster_std=5.0,
                  centers=centroides, shuffle=False, random_state=42)

In [ ]:
X.shape

In [ ]:
from matplotlib  import cm

plt.scatter(X[:,0],X[:,1],c=clases_reales,s=20, marker = 'o', cmap=cm.Set3);

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, homogeneity_completeness_v_measure

In [ ]:
estimador_kmedias = KMeans(n_clusters=3)
etiquetas_kmedias = estimador_kmedias.fit(X).labels_

In [ ]:
plt.scatter(X[:,0],X[:,1], c=etiquetas_kmedias,s=20, marker = 'o', cmap=cm.Set3)
plt.title("Clusters producidos con KMedias");

Dado que en este ejemplo, tenemos las clases verdaderas (ya que hemos generado el dataset de forma artificial), vamos a hacer unas funciones auxiliares que nos ayuden a mostrar de forma clara aquellos puntos que se clasifican de forma incorrecta.

En primer lugar, creamos una función que nos calcule el Índice de Jaccard de dos grupos de puntos (definido como el número de elementos comunes a los dos grupos (intersección) de los dos grupos dividida por el número de elementos en cualquiera de los dos grupos (la unión).

In [ ]:
def jaccard_index(cluster1, cluster2):
    intersection = np.intersect1d(cluster1, cluster2).shape[0]
    union = np.union1d(cluster1, cluster2).shape[0]
    return intersection / union

In [ ]:
jaccard_index(X[clases_reales==0], X[etiquetas_kmedias==0])

Ahora hacemos una funcion, que dado un conjunto de clases y clusters, nos diga que cluster se corresponde a cada clase. Esto es necesario ya que los nombres que da el estimador de clustering no tienen por qué coincidir con las clases reales (es decir, la clase `0` no tiene que representar al mismo número de puntos que el cluster `0`).

In [ ]:
from itertools import product

In [ ]:
list(product([1,2],[3,4]))

In [ ]:
def emparejar_clusters_clases(clusters, clases):
    combinaciones = product(np.unique(clases), np.unique(clusters))
    emparejamientos = {}
    class_ids = np.unique(clases)
    cluster_ids = np.unique(clusters)
    for cluster_id in cluster_ids:
        old_jaccard = 0
        for class_id in class_ids:
            new_jaccard = jaccard_index(X[clases==class_id], X[clusters==cluster_id])
            if new_jaccard > old_jaccard:
                old_jaccard = new_jaccard
                emparejamientos[cluster_id] = class_id
                if new_jaccard>=0.5:
                    break
    return emparejamientos

In [ ]:
emparejar_clusters_clases(etiquetas_kmedias, clases_reales)

Con estas funciones podemos crear una función auxiliar que nos transforma unas etiquetas de cluster para que se correspondan con las clases reales, y una función que nos devuelva aquellas observaciones cuya clase no coincida con el cluster 

In [ ]:
def alinear_clusters(clusters, clases):
    reemplazos = emparejar_clusters_clases(clusters, clases)
    clusters_alineados = [reemplazos[x] for x in clusters]
    return np.array(clusters_alineados)

def encontrar_errores(clusters, clases):
    etiquetas_clusters_alineadas = alinear_clusters(clusters, clases)
    return X[clases!=etiquetas_clusters_alineadas]

Ahora podemos visualizar los puntos mal agrupados

In [ ]:
errores_kmedias = encontrar_errores(etiquetas_kmedias, clases_reales)
plt.scatter(X[:,0],X[:,1],c=etiquetas_kmedias,s=20, marker = 'o', cmap=cm.Set3)
plt.scatter(errores_kmedias[:,0],errores_kmedias[:,1],s=40, marker = 'o', color="red")

plt.title("Clusters producidos con KMedias. Mostrando elementos erróneamente clasificados", size=20);

In [ ]:
silhouette_score(X, etiquetas_kmedias)

In [ ]:
homogeneity_completeness_v_measure(clases_reales, etiquetas_kmedias)

Ahora evaluamos con DBSCAN, es difícil comparar resultados con KMedias dado que en KMedias se especifica el número de clusters y en DBSCAN se especifica la densidad de los mismos

In [ ]:
estimador_dbscan = DBSCAN(eps=10, min_samples=1000)
etiquetas_dbscan = estimador_dbscan.fit(X).labels_

In [ ]:
np.unique(etiquetas_dbscan)

In [ ]:
plt.scatter(X[:,0],X[:,1],c=etiquetas_dbscan,s=20, marker = 'o', cmap=cm.Set3)
errores_dbscan = encontrar_errores(etiquetas_dbscan, clases_reales)
plt.scatter(errores_dbscan[:,0],errores_dbscan[:,1],s=40, marker = 'o', color="red")

plt.title("Clusters producidos con DBSCAN. Mostrando elementos erróneamente clasificados");

In [ ]:
silhouette_score(X, etiquetas_dbscan)

In [ ]:
homogeneity_completeness_v_measure(clases_reales, etiquetas_dbscan)

Vemos que para el caso de tres clusters claramente definidos, KMedias funciona ligeramente mejor que DBSCAN

Probamos ahora con una distribución de datos que no sean tan fáciles de separar en grupos basándose en criterios geométricos.

In [ ]:
from sklearn.datasets import make_circles

la función `make_circles` como su nombre indica, genera un dataset bidimensional en la forma de dos círculos concéntricos

In [ ]:
make_circles?

In [ ]:
X, clases_reales = make_circles(n_samples=5000, factor=.3, noise=.05)

In [ ]:
plt.scatter(X[:,0],X[:,1],c=clases_reales,s=20, marker = 'o', cmap=cm.Set3);

In [ ]:
estimador_kmedias = KMeans(n_clusters=2)
etiquetas_kmedias = estimador_kmedias.fit(X).labels_

In [ ]:
plt.scatter(X[:,0],X[:,1],c=etiquetas_kmedias,s=20, marker = 'o', cmap=cm.Set3)
plt.title("Clusters producidos con KMedias");

Vemos que al basarse en el criterio de cercania al centroide, y dado que ambos clases tienen el mismo centroide, el algoritmo KMedias funciona de forma bastante mediocre, por no decir catastrófica.

In [ ]:
errores_kmedias = encontrar_errores(etiquetas_kmedias, clases_reales)
plt.scatter(X[:,0],X[:,1],c=etiquetas_kmedias,s=20, marker = 'o', cmap=cm.Set3)
plt.scatter(errores_kmedias[:,0],errores_kmedias[:,1],s=40, marker = 'o', color="red")

plt.title("Clusters producidos con KMedias. Mostrando elementos erróneamente clasificados", size=20);

Vemos que KMedias agrupa de forma incorrecta la mitad de los puntos

In [ ]:
silhouette_score(X, etiquetas_kmedias)

In [ ]:
homogeneity_completeness_v_measure(clases_reales, etiquetas_kmedias)

Veamos ahora que tal funciona DBSCAN con este dataset

In [ ]:
estimador_dbscan = DBSCAN(eps=0.05, min_samples=10)
etiquetas_dbscan = estimador_dbscan.fit(X).labels_

In [ ]:
np.unique(etiquetas_dbscan)

In [ ]:
plt.scatter(X[:,0],X[:,1],c=etiquetas_dbscan,s=20, marker = 'o', cmap=cm.Set3)
errores_dbscan = encontrar_errores(etiquetas_dbscan, clases_reales)
plt.scatter(errores_dbscan[:,0],errores_dbscan[:,1],s=40, marker = 'o', color="red")

plt.title("Clusters producidos con DBSCAN. Mostrando elementos erróneamente clasificados");

Vemos que al basarse en un criterio de densidad, DBSCAN encuentra los clusters fácilmente.

In [ ]:
silhouette_score(X, etiquetas_dbscan)

In [ ]:
homogeneity_completeness_v_measure(clases_reales, etiquetas_dbscan)

En este caso he obtenido los hiperparámetros `eps=0.05, min_samples=10` a ojo, iterando sobre el gráfico para ver que valores separaban correctamente las clases reales. En general haríamos una optimización de hiperparámetros para obtener los valores que mejor funcionen.

In [ ]:
estimador_dbscan.predict

Una desventaja del estimador DBSCAN en ´sklearn´ es que no tiene el método predict. Es decir, no puede asignar clusters a nuevos puntos sin tener que ajustar el estimador desde cero.